<a href="https://colab.research.google.com/github/veiro/tesis-imputacion-datos/blob/main/codigo/imputadores/Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LEER
1. Cambiar en enterno a GPU y la variable cuda= True
2. Para ejecutar todo el dataset, MODO_DESARROLLO=False, USAR_GOOGLE_DRIVE =  False y ejecutar un docker local de colab por que demora mas de 12 hrs.
3. tomado de https://zenodo.org/records/10016161

## 1. Requirements

In [ ]:
USAR_GOOGLE_DRIVE =  True
MODO_DESARROLLO = False

In [ ]:
import os
import sys
import random
import numpy as np
import pandas as pd
from math import sqrt

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
import torch.utils.data
import torch.optim as optim
from google.colab import drive
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn import metrics

In [ ]:
if (USAR_GOOGLE_DRIVE):
  from google.colab import drive
  drive.mount('/content/gdrive')
  PATH_DATA_PROCESADA = "/content/gdrive/MyDrive/Maestria/Tesis/Codigo/data/data-procesada"
  PATH_UTILS= "/content/gdrive/MyDrive/Maestria/Tesis/Codigo/utils"
else:
  PATH_DATA_PROCESADA = "C:\\Users\\aveiro\\Desktop\\Tesis\\data\\data-procesada"
  PATH_UTILS = "C:\\Users\\aveiro\\Desktop\\Tesis\\Codigo\\utils\\"

sys.path.append(PATH_UTILS)

import utils as utils
import importlib
importlib.reload(utils)

if (30 ==  utils.version()):
  print("version correcta")
else:
  raise Exception("Version de Util vieja")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
version correcta


In [ ]:
!python3 -m pip freeze

absl-py==1.4.0
aiohttp==3.9.5
aiosignal==1.3.1
alabaster==0.7.16
albumentations==1.3.1
altair==4.2.2
annotated-types==0.6.0
anyio==3.7.1
appdirs==1.4.4
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array_record==0.5.1
arviz==0.15.1
astropy==5.3.4
astunparse==1.6.3
async-timeout==4.0.3
atpublic==4.1.0
attrs==23.2.0
audioread==3.0.1
autograd==1.6.2
Babel==2.14.0
backcall==0.2.0
beautifulsoup4==4.12.3
bidict==0.23.1
bigframes==1.2.0
bleach==6.1.0
blinker==1.4
blis==0.7.11
blosc2==2.0.0
bokeh==3.3.4
bqplot==0.12.43
branca==0.7.1
build==1.2.1
CacheControl==0.14.0
cachetools==5.3.3
catalogue==2.0.10
certifi==2024.2.2
cffi==1.16.0
chardet==5.2.0
charset-normalizer==3.3.2
chex==0.1.86
click==8.1.7
click-plugins==1.1.1
cligj==0.7.2
cloudpathlib==0.16.0
cloudpickle==2.2.1
cmake==3.27.9
cmdstanpy==1.2.2
colorcet==3.1.0
colorlover==0.3.0
colour==0.1.5
community==1.0.0b1
confection==0.1.4
cons==0.4.6
contextlib2==21.6.0
contourpy==1.2.1
cryptography==42.0.5
cufflinks==0.17.3
cupy-cuda12x==12.2.0

## 3. Prepare Data

## Armo juego de datos entramiento y test


In [ ]:
df_train_encoded, df_train_encoded_missing, mask_train_open = utils.obtenerDatosTrain(MODO_DESARROLLO, PATH_DATA_PROCESADA)
df_test_encoded, df_test_encoded_missing, mask_test_open = utils.obtenerDatosTest(MODO_DESARROLLO, PATH_DATA_PROCESADA)

df_test_encoded = pd.DataFrame(df_test_encoded, columns = df_train_encoded.columns)
df_test_encoded_missing = pd.DataFrame(df_test_encoded_missing, columns = df_train_encoded.columns)



In [ ]:
utils.mostrarDatos(df_train_encoded)

Nombre datos: 

--------------------------------------------------------------------------------

Index(['Country_Region_Australia', 'Country_Region_Belgium',
       'Country_Region_Brazil', 'Country_Region_Canada',
       'Country_Region_Chile', 'Country_Region_China',
       'Country_Region_Colombia', 'Country_Region_Denmark',
       'Country_Region_France', 'Country_Region_Germany',
       'Country_Region_India', 'Country_Region_Italy', 'Country_Region_Japan',
       'Country_Region_Malaysia', 'Country_Region_Mexico',
       'Country_Region_Netherlands', 'Country_Region_New Zealand',
       'Country_Region_Pakistan', 'Country_Region_Peru',
       'Country_Region_Russia', 'Country_Region_Spain',
       'Country_Region_Sweden', 'Country_Region_US', 'Country_Region_Ukraine',
       'Country_Region_United Kingdom', 'Lat', 'Long_', 'Confirmed', 'Deaths',
       'Incident_Rate', 'Case_Fatality_Ratio', 'Province_State_encoded',
       'Last_Update_encoded', 'date_encoded'],
      dtype='ob

In [ ]:
%%time
from sklearn.model_selection import train_test_split


def getDataTrain(data, target):

  df_train_encoded_copy =  data.copy()
  Ytrain =  df_train_encoded_copy[target]
  df_train_encoded_copy = df_train_encoded_copy.drop([target], axis=1 )


  ss = StandardScaler()
  ss = ss.fit(df_train_encoded_copy)

  Xtrain0 = ss.transform(df_train_encoded_copy)

  Xtrain = np.nan_to_num(Xtrain0)


  ##测试集
  ss1 = StandardScaler()
  ss1=ss1.fit(Ytrain.values.reshape(-1,1))
  Ytrain=ss1.transform(Ytrain.values.reshape(-1,1))

  return Xtrain, Ytrain, ss1

def getDataTrainEval( data,target, eval_size):
  train, eval = train_test_split(data, test_size=eval_size)
  Xtrain, Ytrain, ssTrain = getDataTrain(train, target)
  Xeval, Yeval, ssEval = getDataTrain(eval, target)
  return Xtrain, Ytrain, ssTrain, Xeval, Yeval , ssEval

def getDataTest(target):

  df_test_encoded_missing_copy =  df_test_encoded_missing.copy()

  Y_DS =   df_test_encoded[target]
  df_test_encoded_missing_copy = df_test_encoded_missing_copy.drop([target], axis=1 )

  ss = StandardScaler()
  ss = ss.fit(df_test_encoded_missing_copy)

  X_DS0 = ss.transform(df_test_encoded_missing_copy)

  X_DS = np.nan_to_num(X_DS0)

  ##测试集
  ss3 = StandardScaler()
  ss3 = ss3.fit(Y_DS.values.reshape(-1,1))
  Y_DS=ss3.transform(Y_DS.values.reshape(-1,1))
  return X_DS, Y_DS, ss3

CPU times: user 9 µs, sys: 0 ns, total: 9 µs
Wall time: 11.2 µs


## 4. Define Model

In [ ]:
class Transformer_block(nn.Module):
    def __init__(self, i_channel, o_channel, mask_radio=0.1, mask=True):
        super(Transformer_block, self).__init__()
        self.Q = nn.Linear(i_channel, o_channel, bias=False)
        self.K = nn.Linear(i_channel, o_channel, bias=False)
        self.V = nn.Linear(i_channel, o_channel)
        self.mask = mask
        self.mask_radio = mask_radio

    def forward(self, input):
        Q_x = self.Q(input)
        K_x = self.K(input)
        V_x = self.V(input)
        weights = torch.matmul(Q_x / np.sqrt(Q_x.shape[-1]), K_x.permute(0, 2, 1))
        if self.mask:
            m_r = torch.ones_like(weights) * self.mask_radio
            weights = weights + torch.bernoulli(m_r) * -1e12
        attention = F.softmax(weights, dim=-1)
        out = torch.matmul(attention, V_x)
        return out + input

class Transformer_model(nn.Module):
    def __init__(self, i_channel):
        super(Transformer_model, self).__init__()
        self.embed1 = nn.Conv1d(in_channels=1, out_channels=128, kernel_size=(1,))#, padding=(1,)
        self.embed2 = nn.Conv1d(in_channels=1, out_channels=128, kernel_size=(1,))
        self.embed3 = nn.Conv1d(in_channels=1, out_channels=128, kernel_size=(1,))

        self.transformer1 = Transformer_block(128, 128, 0.1, True)#nn.ModuleList([Transformer_block(64, 64, True) for _ in range(2)]) #
        self.transformer2 = Transformer_block(128, 128, 0.25, True)
        self.transformer3 = Transformer_block(128, 128, 0.5, True)

        self.Linear_c1 = nn.ModuleList([nn.Linear(128, 128) for _ in range(3)])
        self.Linear_c2 = nn.ModuleList([nn.Linear(128, 128) for _ in range(3)])
        self.Linear_c3 = nn.ModuleList([nn.Linear(128, 128) for _ in range(3)])
        self.act_r = nn.ReLU()
        self.act_g = nn.GELU()
        self.sigmoid = nn.Sigmoid()
        self.Linears1 = nn.ModuleList([nn.Linear(i_channel * 2, i_channel * 2) for _ in range(3)])
        self.Linears2 = nn.ModuleList([nn.Linear(i_channel * 2, i_channel * 2) for _ in range(3)])
        self.Linears3 = nn.ModuleList([nn.Linear(i_channel * 2, i_channel * 2) for _ in range(3)])
        self.drop2 = nn.Dropout(0.15)
        self.drop3 = nn.Dropout(0.3)
        self.output = nn.Linear(i_channel * 2, 1)
        self.output2 = nn.Linear(i_channel * 2, 1)
        self.output3 = nn.Linear(i_channel * 2, 1)

    def forward(self, input):
        x1 = self.embed1(input.unsqueeze(1)).permute(0, 2, 1)
        x2 = self.embed2(input.unsqueeze(1)).permute(0, 2, 1)
        x3 = self.embed3(input.unsqueeze(1)).permute(0, 2, 1)

        for i in range(3):
            x1 = self.act_r(self.Linear_c1[i](x1))
            x2 = self.act_r(self.Linear_c2[i](x2))
            x3 = self.act_r(self.Linear_c3[i](x3))

        x1 = self.transformer1(x1)
        x2 = self.transformer2(x2)
        x3 = self.transformer3(x3)

        x1_1 = torch.mean(x1, dim=2)
        x1_2, _ = torch.max(x1, dim=2)
        x1 = torch.cat((x1_1, x1_2), dim=1)

        x2_1 = torch.mean(x2, dim=2)
        x2_2, _ = torch.max(x2, dim=2)
        x2 = torch.cat((x2_1, x2_2), dim=1)

        x3_1 = torch.mean(x3, dim=2)
        x3_2, _ = torch.max(x3, dim=2)
        x3 = torch.cat((x3_1, x3_2), dim=1)

        for i in range(3):
            x1 = self.act_g(self.Linears1[i](x1))
            x2 = self.act_g(self.Linears2[i](x2))
            x3 = self.act_g(self.Linears3[i](x3))

        out = self.output(x1)
        F2 = self.output2(x2)
        F3 = self.output3(x3)
        return out, F2, F3

    def __call__(self, Xtrain, Ytrain=None, train=True):
        out, F2, F3 = self.forward(Xtrain)
        loss_mse = nn.MSELoss()
        loss_cal = nn.SmoothL1Loss()
        if train:
            # loss = loss_cal(out_last, Ytrain)
            # print(out.shape, Ytrain.shape)
            loss = loss_mse(out, Ytrain)
            loss2 = loss_cal(F2, Ytrain)
            loss3 = loss_cal(F3, Ytrain)

            return loss + loss2 + loss3
        else:
            # print((out + F2 + F3).shape)
            return (out + F2 + F3) / 3 #out * 0.5 + F2 * 0.25 + F3 * 0.25

# R^2 training: 0.874, R^2 test: 0.845,DS R^2 : 0.792

## 6. Train Model

In [ ]:
%%time
import os
import torch.optim as optim
import torch
from tqdm import tqdm
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"
#copair LE_DEEP
use_cuda = True

def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True

device = torch.device("cuda" if use_cuda else "cpu") # v100 GPU


def train(Xtrains, Ytrains, model, lr, wd, max_norm=10, batch_size=32):
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    optimizer.zero_grad()
    N = len(Xtrains)
    loss_total = 0
    Xtrain, Ytrain = [], []
    for i in tqdm(range(N)):
        # print(Xtrains[i])
        Xtrain.append(Xtrains[i])
        Ytrain.append(Ytrains[i])
        if i % batch_size == 0 or i == N:
            X_train = torch.tensor(np.array(Xtrain)).to(device).float()
            Y_train = torch.tensor(np.array(Ytrain)).to(device).float()
            loss = model(X_train, Y_train)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)
            Xtrain, Ytrain = [], []
        else:
            continue
        if i % batch_size == 0 or i == N:
            optimizer.step()
            optimizer.zero_grad()
        loss_total += loss.item()
    return loss_total

def eva(Xtests, model):
    model_eva = model.eval()
    N = len(Xtests)
    Y_pre = []
    for i in tqdm(range(N)):
        X_test = torch.tensor(np.array([Xtests[i]])).to(device).float()
        out = model(X_test, train=False).to('cpu').data.numpy()
        Y_pre.append(out)
    return np.array(Y_pre).reshape(-1,1)


CPU times: user 43 µs, sys: 0 ns, total: 43 µs
Wall time: 45.5 µs


In [ ]:

setup_seed(2023)
size_representation=33
model = Transformer_model(size_representation).to(device) #LSTM_model, GRU_model, Transformer_model
# train
iteration = 10 # 60
lr = 0.0005
wd = 0.01
max_norm=10
batch_size=128 #32
eval_size = 0.01 # 0.1

i =14 #44 - 33 # cambiar este i por cada dia

columnas = df_train_encoded.columns[i:]

for column in columnas:
  model = Transformer_model(size_representation).to(device)
  print("Column: ", column)
  Xtrain, Ytrain, ss1, Xeval, Yeval , ssEval = getDataTrainEval(df_train_encoded, column, eval_size)

  rmse_best = 1000000
  for epoch in range(1, iteration):
      print("Epoch: "+ str(epoch) + " of " + str(iteration))
      if epoch % 2 == 0 and lr >= 0.000025:
          lr /= 2
      loss_train = train(Xtrain, Ytrain, model, lr, wd, max_norm, batch_size)

      Yeval_predict = eva(Xeval, model)
      Yeval_predict = ssEval.inverse_transform(Yeval_predict)
      Yeval = ssEval.inverse_transform(Yeval)
      #Y_DS_predict = eva(X_DS, model)
      #Y_DS_predict = ss1.inverse_transform(Y_DS_predict)

      rmse_train = sqrt(metrics.mean_squared_error(Yeval, Yeval_predict))
      #rmse_test = sqrt(metrics.mean_squared_error(Y_DS, Y_DS_predict))
      print('RMSE training: ', rmse_train)
      if rmse_train < rmse_best:
          rmse_best = rmse_train
          torch.save(model, PATH_DATA_PROCESADA+'/model_csse_covid_19_data_Trasnformer_'+column+'_Desarrollo='+str(MODO_DESARROLLO))
          print("Se guardo el mejor modelo")
          print("------------------------------------------------")
  i = i+1
  print("Termino i=", i)
  print("------------------------------------------------")

print("Learning Finished!")



Column:  Country_Region_Mexico
Epoch: 1 of 10


100%|██████████| 20946/20946 [01:18<00:00, 265.83it/s]


RMSE training:  0.002802338328491681
Se guardo el mejor modelo
------------------------------------------------
Epoch: 2 of 10


100%|██████████| 20946/20946 [01:19<00:00, 263.79it/s]


RMSE training:  0.07849233140021132
Epoch: 3 of 10


...

RMSE training:  7.185948765744251e+16
Epoch: 8 of 10


100%|██████████| 20946/20946 [01:20<00:00, 258.69it/s]


RMSE training:  1.6671710938367216e+19
Epoch: 9 of 10


100%|██████████| 20946/20946 [01:21<00:00, 257.14it/s]


RMSE training:  3.8679088130638e+21
Termino i= 34
------------------------------------------------
Learning Finished!


## Guardo el resusltado

In [ ]:
%%time

columnas = df_train_encoded.columns#[1:]
i = 0

try :
  filled_data = pd.read_csv(PATH_DATA_PROCESADA+'/csse_covid_19_data_Transformer_Desarrollo='+str(MODO_DESARROLLO)+'.csv')
except:
  filled_data = pd.DataFrame()
  pass

for  column in columnas:
  if column in filled_data.columns:
    print("Column procesada: ", column)
    i = i+1
    continue
  else:
    print("Column: ", column)
    X_DS, Y_DS, ss3 = getDataTest(column)
    model = torch.load( PATH_DATA_PROCESADA+'/model_csse_covid_19_data_Trasnformer_'+column+'_Desarrollo='+str(MODO_DESARROLLO))
    #model.eval()
    Y_DS_predict = eva(X_DS, model)
    Y_DS_predict = ss3.inverse_transform(Y_DS_predict)
    Y_DS = ss3.inverse_transform(Y_DS)
    rmse_column =  sqrt(metrics.mean_squared_error(Y_DS, Y_DS_predict))
    print('RMSE column for ', (column),(rmse_column))
    print("------------------------------------------------")
    output_list = [item[0] for item in Y_DS_predict]
    filled_data[column] = output_list
    print("GUARDO EN csv")
    filled_data.to_csv(PATH_DATA_PROCESADA+'/csse_covid_19_data_Transformer_Desarrollo='+str(MODO_DESARROLLO)+'.csv', index=False)
    i = i+1
    print("Termino i=", i)
    print("------------------------------------------------")

print("Se termino las pruebas!")

Column procesada:  Country_Region_Australia
Column procesada:  Country_Region_Belgium
Column procesada:  Country_Region_Brazil
Column procesada:  Country_Region_Canada
Column procesada:  Country_Region_Chile
Column procesada:  Country_Region_China
Column procesada:  Country_Region_Colombia
Column procesada:  Country_Region_Denmark
Column procesada:  Country_Region_France
Column procesada:  Country_Region_Germany
Column procesada:  Country_Region_India
Column procesada:  Country_Region_Italy
Column procesada:  Country_Region_Japan
Column procesada:  Country_Region_Malaysia
Column procesada:  Country_Region_Mexico
Column procesada:  Country_Region_Netherlands
Column procesada:  Country_Region_New Zealand
Column procesada:  Country_Region_Pakistan
Column procesada:  Country_Region_Peru
Column procesada:  Country_Region_Russia
Column procesada:  Country_Region_Spain
Column procesada:  Country_Region_Sweden
Column procesada:  Country_Region_US
Column:  Country_Region_Ukraine


100%|██████████| 897667/897667 [53:35<00:00, 279.17it/s]


RMSE column for  Country_Region_Ukraine 0.08393983779450627
------------------------------------------------
RMSE total  0.08393983779450627
------------------------------------------------
GUARDO EN csv
Termino i= 24
------------------------------------------------
Column:  Country_Region_United Kingdom


100%|██████████| 897667/897667 [53:36<00:00, 279.06it/s]


RMSE column for  Country_Region_United Kingdom 0.14431681031133256
------------------------------------------------
RMSE total  0.22825664810583884
------------------------------------------------
GUARDO EN csv
Termino i= 25
------------------------------------------------
Column:  Lat


...

RMSE column for  Province_State_encoded 148.6064034791094
------------------------------------------------
RMSE total  217484.3825667545
------------------------------------------------
GUARDO EN csv
Termino i= 32
------------------------------------------------
Column:  Last_Update_encoded


100%|██████████| 897667/897667 [50:59<00:00, 293.37it/s]


RMSE column for  Last_Update_encoded 152.02872600692936
------------------------------------------------
RMSE total  217636.41129276142
------------------------------------------------
GUARDO EN csv
Termino i= 33
------------------------------------------------
Column:  date_encoded


100%|██████████| 897667/897667 [50:56<00:00, 293.71it/s]


RMSE column for  date_encoded 173.20791147926502
------------------------------------------------
RMSE total  217809.61920424068
------------------------------------------------
GUARDO EN csv
Termino i= 34
------------------------------------------------
RMSE total test for  0.24263966393355296
Se termino las pruebas!
CPU times: user 9h 33min 12s, sys: 1min 47s, total: 9h 34min 59s
Wall time: 9h 33min 43s


## Calculo RMSE

In [ ]:
filled_data = pd.read_csv(PATH_DATA_PROCESADA+'/csse_covid_19_data_Transformer_Desarrollo='+str(MODO_DESARROLLO)+'.csv')
filled_data.reset_index(drop=True, inplace=True)
filled_data

,Country_Region_Australia,Country_Region_Belgium,Country_Region_Brazil,Country_Region_Canada,Country_Region_Chile,Country_Region_China,Country_Region_Colombia,Country_Region_Denmark,Country_Region_France,Country_Region_Germany,...,Country_Region_United Kingdom,Lat,Long_,Confirmed,Deaths,Incident_Rate,Case_Fatality_Ratio,Province_State_encoded,Last_Update_encoded,date_encoded
0,-1.725459e-04,0.008322,0.002234,0.038947,0.000177,0.007937,-0.000150,0.000212,0.020261,0.000878,...,0.022626,38.634070,-89.636010,12684.9640,390.835200,23088.2560,2.064776,311.47482,529.602540,305.113600
1,7.271792e-06,0.000192,-0.000273,0.001110,-0.000071,-0.000644,-0.000247,0.000014,0.000399,-0.000326,...,0.001299,38.577446,-86.995735,1421.3782,56.796450,24491.3670,1.514999,371.50146,405.461300,207.482850
2,-1.172346e-05,0.000047,-0.000100,0.002781,0.000131,-0.000713,-0.000359,0.000056,0.000113,-0.000487,...,-0.002449,36.280030,-91.716870,49159.6950,585.362400,26266.6910,0.511323,353.51822,567.942750,221.050160
3,-8.800903e-05,0.000770,0.001102,0.001915,0.000303,0.001312,-0.000217,-0.000219,0.002565,-0.000445,...,0.004929,31.772310,-95.355576,7858.3780,8.590275,6628.6416,1.864329,361.46182,-18.666399,9.746195
4,-5.922324e-05,0.016164,0.001850,0.041086,0.000132,0.009330,-0.000011,0.000035,0.026773,0.002974,...,0.023477,38.691772,-90.667700,20604.2300,189.833680,18782.0840,1.599989,332.52734,290.975900,120.510260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
897662,-2.165439e-05,0.000392,0.000096,0.000706,-0.000244,0.000098,-0.000201,0.000141,0.001306,-0.000385,...,0.002313,37.259335,-89.092000,62401.9960,614.096900,9188.8200,1.621869,345.39655,130.981490,275.505980
897663,-1.279528e-04,0.011986,0.001993,0.059711,0.000322,0.007905,0.000017,0.001367,0.019084,-0.000057,...,0.025195,40.614662,-97.863365,35858.5350,324.133420,8350.0600,1.575693,349.01242,408.069430,178.301030
897664,-9.230476e-05,0.000906,0.000496,0.002454,-0.000024,0.000039,-0.000505,-0.000017,0.001027,0.001786,...,0.002225,38.829765,-101.446690,6719.0770,321.037170,22246.3770,0.934488,405.26780,550.357800,305.939100
897665,-3.452294e-05,0.000381,0.000091,0.000796,0.000133,0.000402,-0.000220,-0.000196,-0.000378,-0.000039,...,0.000284,36.570770,-89.689316,84997.5600,962.317700,9925.2270,1.228095,293.95468,366.553770,361.204250


In [ ]:
rmse = utils.rmse(df_test_encoded.to_numpy(), filled_data.to_numpy(), mask_test_open)
print("rmse :", rmse)

rmse : 15137.151001106482


In [ ]:
df_test_encoded, filled_data = utils.obtener_df_imputados(df_test_encoded, filled_data, mask_test_open)

In [ ]:
print("inicio de decoding")

imputed_data_decoded=utils.decode(filled_data, df_train_encoded, PATH_DATA_PROCESADA)

print("imputed_data_decoded.head(5)")
print(imputed_data_decoded.head(5))

print("imputed_data_decoded.to_csv")
imputed_data_decoded.to_csv(PATH_DATA_PROCESADA+'/csse_covid_19_data_Transformer_final_Desarrollo='+str(MODO_DESARROLLO)+'.csv', index=False)

print("fin de decoding")

inicio de decoding


897667it [01:52, 7960.39it/s]


imputed_data_decoded.head(5)
  Province_State Country_Region          Last_Update        Lat    Long_  \
0          Texas             US  2021-01-02 05:22:33  41.123513 -83.1278   
1          Texas             US  2021-01-02 05:22:33  38.577446 -75.2996   
2          Texas             US  2021-01-02 05:22:33  35.780773 -91.7169   
3          Texas             US  2021-01-02 05:22:33  33.618867 -95.3556   
4          Texas             US  2021-01-02 05:22:33  43.720997 -87.9458   

   Confirmed  Deaths  Incident_Rate  Case_Fatality_Ratio        date  
0      12750     230   23107.035413             1.788235  01-01-2021  
1      10041     167   19549.853001             1.663181  01-01-2021  
2      49159     406   30784.732824             1.325795  01-01-2021  
3        543      27    4167.013225             5.389222  01-01-2021  
4      15026     171   13027.570661             1.138027  01-01-2021  
imputed_data_decoded.to_csv
fin de decoding
